# AoC 10

The CPU has a single register, X, which starts with the value 1. 

It supports only two instructions:

* addx V takes two cycles to complete. After two cycles, the X register is increased by the value V. (V can be negative.)
* noop takes one cycle to complete. It has no other effect.

Consider the following small program:

* noop
* addx 3
* addx -5

Execution of this program proceeds as follows:

At the start of the first cycle, the noop instruction begins execution. \
During the first cycle, X is 1. \
After the first cycle, the noop instruction finishes execution, doing nothing.

At the start of the second cycle, the addx 3 instruction begins execution. \
During the second cycle, X is still 1.\
During the third cycle, X is still 1. \
After the third cycle, the addx 3 instruction finishes execution, setting X to 4.

At the start of the fourth cycle, the addx -5 instruction begins execution. \
During the fourth cycle, X is still 4.\
During the fifth cycle, X is still 4. \
After the fifth cycle, the addx -5 instruction finishes execution, setting X to -1.

The signal strength (the cycle number multiplied by the value of the X register) 
during the 20th cycle and every 40 cycles after that (that is, during the 20th, 60th, 100th, 140th, 180th, and 220th cycles)

in the test 2 the interesting signal strengths can be determined as follows:

- During the 20th cycle, register X has the value 21, so the signal strength is 20 * 21 = 420. (The 20th cycle occurs in the middle of the second addx -1, so the value of register X is the starting value, 1, plus all of the other addx values up to that point: 1 + 15 - 11 + 6 - 3 + 5 - 1 - 8 + 13 + 4 = 21.)\
- During the 60th cycle, register X has the value 19, so the signal strength is 60 * 19 = 1140.\
- During the 100th cycle, register X has the value 18, so the signal strength is 100 * 18 = 1800.\
- During the 140th cycle, register X has the value 21, so the signal strength is 140 * 21 = 2940.\
- During the 180th cycle, register X has the value 16, so the signal strength is 180 * 16 = 2880.\
- During the 220th cycle, register X has the value 18, so the signal strength is 220 * 18 = 3960.\

The sum of these signal strengths is 13140.

Find the signal strength during the 20th, 60th, 100th, 140th, 180th, and 220th cycles. What is the sum of these six signal strengths?

In [1]:
def get_input(filename):
    instructions = []
    with open(filename) as file:
        for line in file:            
            instructions.append(line.strip())
    return instructions

In [2]:
def acummulated_strength(instructions, verbose=False):
    val = 1
    cycle = 0
    acum = 0
    for inst in instructions:
        if inst.startswith('noop'):
            cycle += 1
            if cycle == 20 or (cycle - 20) % 40 == 0:
                print(f'({inst}) cycle: {cycle } val: {val} strength: {cycle * val}')
                acum += cycle * val
            if verbose:
                print(f'cycle: {cycle} = {val}')
        else:
            istr, xstr = inst.split()
            x = int(xstr)
            if cycle + 1 == 20 or (cycle + 1 - 20) % 40 == 0:
                print(f'({inst}) cycle: {cycle + 1} val: {val} strength: {(cycle + 1) * val}')
                acum += (cycle + 1) * val
                if verbose:
                    print(f'cycle: {cycle + 1} = {val}')
            elif cycle + 2 == 20 or (cycle + 2 - 20) % 40 == 0:
                print(f'({inst}) cycle: {cycle + 2} val: {val} strength: {(cycle + 2) * val}')
                acum += (cycle + 2) * val
                if verbose:
                    print(f'cycle: {cycle + 2} = {val}')
            val += x
            cycle += 2
            if verbose:
                print(f'cycle: {cycle} = {val}')
    return acum

In [3]:
instructions = get_input('test1.txt')
strength = acummulated_strength(instructions, verbose=True)
strength

cycle: 1 = 1
cycle: 3 = 4
cycle: 5 = -1


0

In [4]:
instructions = get_input('test2.txt')
strength = acummulated_strength(instructions, verbose=False)
strength

(addx -1) cycle: 20 val: 21 strength: 420
(addx -3) cycle: 60 val: 19 strength: 1140
(noop) cycle: 100 val: 18 strength: 1800
(addx 1) cycle: 140 val: 21 strength: 2940
(addx -9) cycle: 180 val: 16 strength: 2880
(addx 1) cycle: 220 val: 18 strength: 3960


13140

In [5]:
instructions = get_input('input.txt')
strength = acummulated_strength(instructions, verbose=False)
strength

(addx 17) cycle: 20 val: 21 strength: 420
(addx 2) cycle: 60 val: 17 strength: 1020
(addx 17) cycle: 100 val: 4 strength: 400
(addx 3) cycle: 140 val: 21 strength: 2940
(addx -10) cycle: 180 val: 21 strength: 3780
(addx 11) cycle: 220 val: 21 strength: 4620


13180

### Part 2

It seems like the X register controls the horizontal position of a sprite. Specifically, the sprite is 3 pixels wide, and the X register sets the horizontal position of the middle of that sprite. (In this system, there is no such thing as "vertical position": if the sprite's horizontal position puts its pixels where the CRT is currently drawing, then those pixels will be drawn.)

You count the pixels on the CRT: 40 wide and 6 high. This CRT screen draws the top row of pixels left-to-right, then the row below that, and so on. The left-most pixel in each row is in position 0, and the right-most pixel in each row is in position 39.

Like the CPU, the CRT is tied closely to the clock circuit: the CRT draws a single pixel during each cycle. Representing each pixel of the screen as a #, here are the cycles during which the first and last pixel in each row are drawn:

Cycle   1 -> ######################################## <- Cycle  40 <br>
Cycle  41 -> ######################################## <- Cycle  80 <br>
Cycle  81 -> ######################################## <- Cycle 120 <br>
Cycle 121 -> ######################################## <- Cycle 160 <br>
Cycle 161 -> ######################################## <- Cycle 200 <br>
Cycle 201 -> ######################################## <- Cycle 240 <br>

So, by carefully timing the CPU instructions and the CRT drawing operations, you should be able to determine whether the sprite is visible the instant each pixel is drawn. If the sprite is positioned such that one of its three pixels is the pixel currently being drawn, the screen produces a lit pixel (#); otherwise, the screen leaves the pixel dark (.).

The first few pixels from the larger example above are drawn as follows:

Sprite position: ###..................................... <br>

Start cycle   1: begin executing addx 15 <br>
During cycle  1: CRT draws pixel in position 0 <br>
Current CRT row: # <br>

During cycle  2: CRT draws pixel in position 1 <br>
Current CRT row: ## <br>
End of cycle  2: finish executing addx 15 (Register X is now 16) <br>
Sprite position: ...............###...................... <br>

Start cycle   3: begin executing addx -11 <br>
During cycle  3: CRT draws pixel in position 2 <br>
Current CRT row: ##. <br>

During cycle  4: CRT draws pixel in position 3 <br>
Current CRT row: ##.. <br>
End of cycle  4: finish executing addx -11 (Register X is now 5) <br>
Sprite position: ....###................................. <br>

Start cycle   5: begin executing addx 6 <br>
During cycle  5: CRT draws pixel in position 4 <br>
Current CRT row: ##..# <br>

During cycle  6: CRT draws pixel in position 5 <br>
Current CRT row: ##..## <br>
End of cycle  6: finish executing addx 6 (Register X is now 11) <br>
Sprite position: ..........###........................... <br>

Start cycle   7: begin executing addx -3 <br>
During cycle  7: CRT draws pixel in position 6 <br>
Current CRT row: ##..##. <br>

During cycle  8: CRT draws pixel in position 7 <br>
Current CRT row: ##..##.. <br>
End of cycle  8: finish executing addx -3 (Register X is now 8) <br>
Sprite position: .......###.............................. <br>

Start cycle   9: begin executing addx 5 <br>
During cycle  9: CRT draws pixel in position 8 <br>
Current CRT row: ##..##..# <br>

During cycle 10: CRT draws pixel in position 9 <br>
Current CRT row: ##..##..## <br>
End of cycle 10: finish executing addx 5 (Register X is now 13) <br>
Sprite position: ............###......................... <br>

Start cycle  11: begin executing addx -1 <br>
During cycle 11: CRT draws pixel in position 10 <br>
Current CRT row: ##..##..##. <br>

During cycle 12: CRT draws pixel in position 11 <br>
Current CRT row: ##..##..##.. <br>
End of cycle 12: finish executing addx -1 (Register X is now 12) <br>
Sprite position: ...........###.......................... <br>

Start cycle  13: begin executing addx -8 <br>
During cycle 13: CRT draws pixel in position 12 <br>
Current CRT row: ##..##..##..# <br>

During cycle 14: CRT draws pixel in position 13 <br>
Current CRT row: ##..##..##..## <br>
End of cycle 14: finish executing addx -8 (Register X is now 4) <br>
Sprite position: ...###.................................. <br>

Start cycle  15: begin executing addx 13 <br>
During cycle 15: CRT draws pixel in position 14 <br>
Current CRT row: ##..##..##..##. <br>

During cycle 16: CRT draws pixel in position 15 <br>
Current CRT row: ##..##..##..##.. <br>
End of cycle 16: finish executing addx 13 (Register X is now 17) <br>
Sprite position: ................###..................... <br>

Start cycle  17: begin executing addx 4 <br>
During cycle 17: CRT draws pixel in position 16 <br>
Current CRT row: ##..##..##..##..# <br>

During cycle 18: CRT draws pixel in position 17 <br>
Current CRT row: ##..##..##..##..## <br>
End of cycle 18: finish executing addx 4 (Register X is now 21) <br>
Sprite position: ....................###................. <br>

Start cycle  19: begin executing noop <br>
During cycle 19: CRT draws pixel in position 18 <br>
Current CRT row: ##..##..##..##..##. <br>
End of cycle 19: finish executing noop <br>

Start cycle  20: begin executing addx -1 <br>
During cycle 20: CRT draws pixel in position 19 <br>
Current CRT row: ##..##..##..##..##.. <br>

During cycle 21: CRT draws pixel in position 20 <br>
Current CRT row: ##..##..##..##..##..# <br>
End of cycle 21: finish executing addx -1 (Register X is now 20) <br>
Sprite position: ...................###.................. <br>

Allowing the program to run to completion causes the CRT to produce the following image:

##..##..##..##..##..##..##..##..##..##.. <br>
###...###...###...###...###...###...###. <br>
####....####....####....####....####.... <br>
#####.....#####.....#####.....#####..... <br>
######......######......######......#### <br>
#######.......#######.......#######..... <br>

Render the image given by your program. What eight capital letters appear on your CRT?

In [93]:
def print_sprite_pos(crt_col):
    print('Sprite position: ', end='')
    for i in range(40):
        if i in [crt_col-1, crt_col, crt_col+1]:
            print('#', end='')
        else:
            print('.', end='')
    print('')
    
def run_cycle(x, cycle, inst, crt_col, crt_text, full_text, updx):
    if not inst.startswith('noop'):
        istr, xstr = inst.split()
    
    print_sprite_pos(x)
    print(f'Start cycle\t{cycle}: begin executing {inst}')
    print(f'During cycle\t{cycle}: CRT draws pixel in position {crt_col}')            
    if crt_col in [x-1, x, x+1]:
        crt_text += '#'
    else:
        crt_text += '.'            

    print(f'Current CRT row: {crt_text}')
    
    if updx:
        x += int(xstr)
        
    print(f'End of cycle\t{cycle}: finish executing {inst} (Register X is now {x})\n')
    cycle += 1
    crt_col += 1
    
    if crt_col % 40 == 0:
        full_text.append(crt_text)
        crt_col = 0
        crt_text = ''
        
    return x, crt_col, crt_text, cycle
        
def draw_screen(instructions, verbose=False):
    x = 1
    crt_col = 0
    cycle = 1
    crt_text = ''    
    full_text = []
    
    for inst in instructions:
        if inst.startswith('noop'):
            x, crt_col, crt_text, cycle = run_cycle(x, cycle, inst, crt_col, crt_text, full_text, False)
        else:
            #cycle 1
            x, crt_col, crt_text, cycle = run_cycle(x, cycle, inst, crt_col, crt_text, full_text, False)
            
            #cycle 2
            x, crt_col, crt_text, cycle = run_cycle(x, cycle, inst, crt_col, crt_text, full_text, True)
    
    return full_text

In [94]:
instructions = get_input('test2.txt')
w = draw_screen(instructions, verbose=True)
print('\n'.join(w))

Sprite position: ###.....................................
Start cycle	1: begin executing addx 15
During cycle	1: CRT draws pixel in position 0
Current CRT row: #
End of cycle	1: finish executing addx 15 (Register X is now 1)

Sprite position: ###.....................................
Start cycle	2: begin executing addx 15
During cycle	2: CRT draws pixel in position 1
Current CRT row: ##
End of cycle	2: finish executing addx 15 (Register X is now 16)

Sprite position: ...............###......................
Start cycle	3: begin executing addx -11
During cycle	3: CRT draws pixel in position 2
Current CRT row: ##.
End of cycle	3: finish executing addx -11 (Register X is now 16)

Sprite position: ...............###......................
Start cycle	4: begin executing addx -11
During cycle	4: CRT draws pixel in position 3
Current CRT row: ##..
End of cycle	4: finish executing addx -11 (Register X is now 5)

Sprite position: ....###.................................
Start cycle	5: begin execu

Sprite position: ........###.............................
Start cycle	86: begin executing noop
During cycle	86: CRT draws pixel in position 5
Current CRT row: ####..
End of cycle	86: finish executing noop (Register X is now 9)

Sprite position: ........###.............................
Start cycle	87: begin executing noop
During cycle	87: CRT draws pixel in position 6
Current CRT row: ####...
End of cycle	87: finish executing noop (Register X is now 9)

Sprite position: ........###.............................
Start cycle	88: begin executing noop
During cycle	88: CRT draws pixel in position 7
Current CRT row: ####....
End of cycle	88: finish executing noop (Register X is now 9)

Sprite position: ........###.............................
Start cycle	89: begin executing addx 2
During cycle	89: CRT draws pixel in position 8
Current CRT row: ####....#
End of cycle	89: finish executing addx 2 (Register X is now 9)

Sprite position: ........###.............................
Start cycle	90: begi

Sprite position: ............###.........................
Start cycle	172: begin executing addx -1
During cycle	172: CRT draws pixel in position 11
Current CRT row: ######......
End of cycle	172: finish executing addx -1 (Register X is now 12)

Sprite position: ...........###..........................
Start cycle	173: begin executing addx 3
During cycle	173: CRT draws pixel in position 12
Current CRT row: ######......#
End of cycle	173: finish executing addx 3 (Register X is now 12)

Sprite position: ...........###..........................
Start cycle	174: begin executing addx 3
During cycle	174: CRT draws pixel in position 13
Current CRT row: ######......##
End of cycle	174: finish executing addx 3 (Register X is now 15)

Sprite position: ..............###.......................
Start cycle	175: begin executing addx 1
During cycle	175: CRT draws pixel in position 14
Current CRT row: ######......###
End of cycle	175: finish executing addx 1 (Register X is now 15)

Sprite position: ...

In [95]:
instructions = get_input('input.txt')
w = draw_screen(instructions, verbose=True)
print('\n'.join(w))

Sprite position: ###.....................................
Start cycle	1: begin executing addx 2
During cycle	1: CRT draws pixel in position 0
Current CRT row: #
End of cycle	1: finish executing addx 2 (Register X is now 1)

Sprite position: ###.....................................
Start cycle	2: begin executing addx 2
During cycle	2: CRT draws pixel in position 1
Current CRT row: ##
End of cycle	2: finish executing addx 2 (Register X is now 3)

Sprite position: ..###...................................
Start cycle	3: begin executing addx 3
During cycle	3: CRT draws pixel in position 2
Current CRT row: ###
End of cycle	3: finish executing addx 3 (Register X is now 3)

Sprite position: ..###...................................
Start cycle	4: begin executing addx 3
During cycle	4: CRT draws pixel in position 3
Current CRT row: ####
End of cycle	4: finish executing addx 3 (Register X is now 6)

Sprite position: .....###................................
Start cycle	5: begin executing addx 1
Du

Sprite position: .......................................#
Start cycle	85: begin executing addx -32
During cycle	85: CRT draws pixel in position 4
Current CRT row: ###..
End of cycle	85: finish executing addx -32 (Register X is now 40)

Sprite position: .......................................#
Start cycle	86: begin executing addx -32
During cycle	86: CRT draws pixel in position 5
Current CRT row: ###...
End of cycle	86: finish executing addx -32 (Register X is now 8)

Sprite position: .......###..............................
Start cycle	87: begin executing addx -5
During cycle	87: CRT draws pixel in position 6
Current CRT row: ###....
End of cycle	87: finish executing addx -5 (Register X is now 8)

Sprite position: .......###..............................
Start cycle	88: begin executing addx -5
During cycle	88: CRT draws pixel in position 7
Current CRT row: ###....#
End of cycle	88: finish executing addx -5 (Register X is now 3)

Sprite position: ..###...................................

Sprite position: ....###.................................
Start cycle	168: begin executing addx 6
During cycle	168: CRT draws pixel in position 7
Current CRT row: #....#..
End of cycle	168: finish executing addx 6 (Register X is now 5)

Sprite position: ....###.................................
Start cycle	169: begin executing addx 6
During cycle	169: CRT draws pixel in position 8
Current CRT row: #....#...
End of cycle	169: finish executing addx 6 (Register X is now 11)

Sprite position: ..........###...........................
Start cycle	170: begin executing addx -10
During cycle	170: CRT draws pixel in position 9
Current CRT row: #....#....
End of cycle	170: finish executing addx -10 (Register X is now 11)

Sprite position: ..........###...........................
Start cycle	171: begin executing addx -10
During cycle	171: CRT draws pixel in position 10
Current CRT row: #....#....#
End of cycle	171: finish executing addx -10 (Register X is now 1)

Sprite position: ###...............